In [ ]:
#Linear Classification and logistic Regression

In [1]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Hp\Hamoye\NFA 2019 public_data.csv')
print(df)

       country  year  country_code        record     crop_land  grazing_land  \
0      Armenia  1992             1    AreaPerCap  1.402924e-01  1.995463e-01   
1      Armenia  1992             1     AreaTotHA  4.830000e+05  6.870000e+05   
2      Armenia  1992             1  BiocapPerCap  1.598044e-01  1.352610e-01   
3      Armenia  1992             1  BiocapTotGHA  5.501762e+05  4.656780e+05   
4      Armenia  1992             1  EFConsPerCap  3.875102e-01  1.894622e-01   
...        ...   ...           ...           ...           ...           ...   
72181    World  2016          5001  BiocapTotGHA  3.984702e+09  1.504757e+09   
72182    World  2016          5001  EFConsPerCap  5.336445e-01  1.402092e-01   
72183    World  2016          5001  EFConsTotGHA  3.984702e+09  1.046937e+09   
72184    World  2016          5001  EFProdPerCap  5.336445e-01  1.402092e-01   
72185    World  2016          5001  EFProdTotGHA  3.984702e+09  1.046937e+09   

        forest_land  fishing_ground  bu

C:\Users\Hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df['QScore'].value_counts()
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [3]:
#dropping rows with missing values
df = df.dropna()
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [4]:
df[ 'QScore' ] = df[ 'QScore' ].replace([ '1A' ], '2A' ) 
df.QScore.value_counts() 

3A    51473
2A      240
Name: QScore, dtype: int64

In [5]:
df_2A = df[df.QScore== '2A' ] 
df_3A = df[df.QScore== '3A' ].sample( 350 ) 
data_df = df_2A.append(df_3A) 

In [6]:
import sklearn.utils 
data_df = sklearn.utils.shuffle(data_df) 
data_df = data_df.reset_index(drop= True ) 
data_df.shape 
data_df.QScore.value_counts() 


3A    350
2A    240
Name: QScore, dtype: int64

In [7]:
#more preprocessing

data_df = data_df.drop(columns=[ 'country_code' , 'country' , 'year' ]) 
X = data_df.drop(columns= 'QScore' ) 
y = data_df[ 'QScore' ] 

In [8]:
#split the data into training and testing sets

from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 , random_state= 0 ) 
y_train.value_counts() 

3A    239
2A    174
Name: QScore, dtype: int64

In [9]:
#Because of the Imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this. 

#encode categorical variable 

from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder() 
x_train.record = encoder.fit_transform(x_train.record) 
x_test.record = encoder.transform(x_test.record)

C:\Users\Hp\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [10]:
import imblearn 
from imblearn.over_sampling import SMOTE 
smote = SMOTE(random_state= 1 ) 

In [11]:
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train) 

In [12]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler() 
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=[ 'record' ])) 
normalised_train_df = pd.DataFrame(normalised_train_df, 
columns=x_train_balanced.drop(columns=[ 'record' ]).columns) 
normalised_train_df[ 'record' ] = x_train_balanced[ 'record' ] 

In [13]:
x_test = x_test.reset_index(drop= True ) 
normalised_test_df = scaler.transform(x_test.drop(columns=[ 'record' ])) 
normalised_test_df = pd.DataFrame(normalised_test_df, 
columns=x_test.drop(columns=[ 'record' ]).columns) 
normalised_test_df[ 'record' ] = x_test[ 'record' ] 

In [14]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression 
log_reg = LogisticRegression() 
log_reg.fit(normalised_train_df, y_balanced) 

#returns 
LogisticRegression(C= 1.0 , class_weight= None , dual= False , fit_intercept= True , 
 intercept_scaling= 1 , l1_ratio= None , max_iter= 100 , 
 multi_class= 'auto' , n_jobs= None , penalty= 'l2' , 
 random_state= None , solver= 'lbfgs' , tol= 0.0001 , verbose= 0 , 
 warm_start= False )

LogisticRegression()

In [ ]:
# LESSON 2
Measuring Classification Performance

#Cross-Validation and Accuracy 

In [ ]:
from sklearn.model_selection import cross_val_score 
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv= 5 , scoring= 'f1_macro' ) 
scores

In [ ]:
# K-Fold Cross Validation 
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_score = []

In [ ]:
#run for every split
for train_index, test_index in kf.split(normalised_train_df):
    x_train, x_test = normalised_train_df.iloc[train_index],
    normalised_train_df.iloc[test_index]
    
    y_train, y_test = y_balanced[train_index],
    y_balanced[test_index]

model = LogisticRegression().fit(x_train,y_train)

In [ ]:
#save result to list 
f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test),pos_label='2A')*100)

In [ ]:
# Stratified K-Fold Cross Validation 
from sklearn.model_selection import StratifiedKKold
skf = StratifiedKFold(n_splits=5, shuffle=true, random_state=1)
f1_scores = []

In [ ]:
#run for every split 
for train_index,test_index in skf.split(normalised_train_df,y_balanced):
x_train, x_test = np.array(normalised_train_df)[train_index],
                  np.array(normalised_train_df)[test_index]
y_train, y_test = y_balanced[train_index],
                  y_balanced[test_index]

model = LogisticRegression().fit(x_train, y_train)

In [ ]:
#save result to list 
f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A'))

In [ ]:
#Leave One Out Cross Validation (LOOCV)
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegresion(), normalised_train_df, y_balanced, cv=100, scoring='f1_macro')
avarage_score = scores.mean() * 100


In [16]:
#Confusion Matrix
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions,labels= ['2A', '3A'])
cnf_mat

array([[29, 37],
       [50, 61]], dtype=int64)

In [17]:
#Accuracy 
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print('Accuracy:{}'.format(round(accuracy*100), 2))



Accuracy:51


In [18]:
#Precision 
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A')
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 37


In [19]:
#Recall 
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' ) 
print( 'Recall: {}' .format(round(recall* 100 ), 2 )) 

Recall: 44


In [20]:
#F1-Score 
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' ) 
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 40


In [1]:
from sklearn.tree import DecisionTreeclassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

ImportError: cannot import name 'DecisionTreeclassifier' from 'sklearn.tree' (C:\Users\Hp\anaconda3\lib\site-packages\sklearn\tree\__init__.py)